In [63]:
import numpy as np
import math
import csv
csv_reader = csv.reader(open("Mediapipe_world_front.csv"))

with open('csv_file.csv', 'w') as f:
    writer = csv.writer(f)
    for file in csv_reader:

        count +=1
        P_hip=[file[0],file[1],file[2]];  
        P_ne=[file[3],file[4],file[5]];  
        P_sh_l=[file[6],file[7],file[8]]; 
        P_shoulder=[file[9],file[10],file[11]];  
        P_elbow=[file[12],file[13],file[14]];  
        P_wrist=[file[15],file[16],file[17]];    
# 转换成float
        P_hip=[float(P_hip[i]) for i in range(len(P_hip))];
        P_ne=[float(P_ne[i]) for i in range(len(P_ne))];
        P_sh_l=[float(P_sh_l[i]) for i in range(len(P_sh_l))];
        P_shoulder=[float(P_shoulder[i]) for i in range(len(P_shoulder))];
        P_elbow=[float(P_elbow[i]) for i in range(len(P_elbow))];
        P_wrist=[float(P_wrist[i]) for i in range(len(P_wrist))];


#前面的都是导入数据
# 后面的向量都是每帧的向量
        P_se = np.asarray([float(P_elbow[i]) - float(P_shoulder[i]) for i in range(len(P_shoulder))]);
        P_sw = np.asarray([float(P_wrist[i]) - float(P_shoulder[i]) for i in range(len(P_shoulder))]);
        
        P_lr = np.asarray([float(P_shoulder[i]) - float(P_sh_l[i]) for i in range(len(P_shoulder))]);
        P_ew = np.asarray([float(P_wrist[i]) - float(P_elbow[i]) for i in range(len(P_shoulder))]);
        B = np.asarray([float(P_wrist[i]) - float(P_elbow[i]) for i in range(len(P_shoulder))]);
        A = np.asarray([float(P_shoulder[i]) - float(P_elbow[i]) for i in range(len(P_shoulder))]);

   #     print(count)


        x_axis= P_lr/np.sqrt(P_lr.dot(P_lr))
        y_axis=np.cross(x_axis,np.asarray([P_hip[i] - P_shoulder[i] for i in range(len(P_shoulder))]))
        y_axis= y_axis/np.sqrt(y_axis.dot(y_axis)) 
        z_axis=np.cross(x_axis,y_axis);
        z_axis= z_axis/np.sqrt(z_axis.dot(z_axis))

        TT=np.array([x_axis,y_axis,z_axis])

        P_se2=np.dot(TT,P_se)
        
        beta_shoulder = math.atan2(P_se2[2],math.sqrt(P_se2[1]**2+P_se2[0]**2))+ math.pi/2; 
        if P_se2[1]<0:
            beta_shoulder = -beta_shoulder
        
            
        elbow_ang=math.acos(np.dot(A / np.linalg.norm(A),B / np.linalg.norm(B)))
        elbow_d=elbow_ang * 180 / math.pi
        
        beta_d = beta_shoulder * 180 / math.pi
        
        if beta_d < 1:
            P_sw2=np.dot(TT,P_sw)
            alpha_shoulder = math.atan2(P_sw2[1],P_sw2[0])-math.pi/2
            gamma_shoulder=0
        else:
            alpha_shoulder = math.atan2(P_se2[1],P_se2[0])-math.pi/2
            
            P_wrist = np.dot(TT,P_wrist)
            P_elbow = np.dot(TT,P_elbow)
            len_f = np.linalg.norm(B)
            
            if beta_shoulder ==0 or elbow_ang==0:
                gamma_shoulder=0
          
                gamma_d=0
           
            else:
                a=(math.cos(beta_shoulder)*math.cos(elbow_ang)-((P_wrist[2]-P_elbow[2])/len_f))/math.sin(beta_shoulder)/math.sin(elbow_ang)
 
                if a>1 or a<-1:
                    gamma_shoulder= math.pi/2
                else:
                    gamma_shoulder= math.acos(a)-math.pi
                
                P_ew=np.dot(TT,P_ew)
                ang3 = math.atan2(P_ew[1],P_ew[0])
        
                if ang3>(alpha_shoulder+math.pi/2):
                    gamma_shoulder=-gamma_shoulder

        alpha_d= alpha_shoulder * 180 / math.pi               
        gamma_d = gamma_shoulder* 180 / math.pi 
            
            
        row=[alpha_d,beta_d,gamma_d,elbow_d]
        writer.writerow(row)
 



输出：
角度
alpha_d,beta_d,gamma_d,elbow_d

弧度
alpha_shoulder  beta_shoulder   gamma_shoulder  elbow_ang
